# Task 1

In [1]:
# importing modules and libraries

import glob
from pdf2docx import Converter
import docx
import shutil
import os
import re
from elasticsearch import Elasticsearch
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [2]:
#function for converting pdf to docx

def pdf_to_docx(pdf_file):
    docx_file = pdf_file[:-4]+".docx"
    # Create a PDF to DOCX converter object
    cv = Converter(pdf_file)
    # Convert PDF to DOCX
    cv.convert(docx_file, start=0, end=None)
    # Close the converter
    cv.close()

In [3]:
for file in glob.glob('./pdfs/*.pdf'):
    print(file)

./pdfs\15_Nazneen.pdf
./pdfs\1_Ramırez-Duque_.pdf
./pdfs\22_Ouss_ASD.pdf
./pdfs\Abbas_2018.pdf
./pdfs\Abbas_2020.pdf
./pdfs\Asd_Cry_patterns.pdf
./pdfs\carpenter2020 (1).pdf
./pdfs\Dawson.pdf
./pdfs\LEE.pdf
./pdfs\Patten_Audio.pdf
./pdfs\Qiu.pdf
./pdfs\Tariq2018.pdf
./pdfs\Tariq_2019.pdf
./pdfs\Young_Behavior.pdf
./pdfs\zhao2020.pdf


In [4]:
#Converting pdf files to docx files

for file in glob.glob('./pdfs/*.pdf')[1:-3]:
    try:
        pdf_to_docx(file)
    except Exception:
        print('')
for file in glob.glob('./pdfs/*.pdf')[-2:]:
    try:
        pdf_to_docx(file)
    except Exception:
        print('')

[INFO] Start to convert ./pdfs\1_Ramırez-Duque_.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...
[INFO] Start to convert ./pdfs\22_Ouss_ASD.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...


[INFO] [3/4] Parsing pages...
[INFO] (1/7) Page 1
[INFO] (2/7) Page 2
[INFO] (3/7) Page 3
[INFO] (4/7) Page 4
[INFO] (5/7) Page 5
[INFO] (6/7) Page 6
[INFO] (7/7) Page 7
[INFO] [4/4] Creating pages...
[INFO] (1/7) Page 1
[INFO] (2/7) Page 2
[INFO] (3/7) Page 3
[INFO] (4/7) Page 4
[INFO] (5/7) Page 5
[INFO] (6/7) Page 6
[INFO] (7/7) Page 7
[INFO] Terminated in 62.28s.
[INFO] Start to convert ./pdfs\Abbas_2018.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...
[INFO] [3/4] Parsing pages...
[INFO] (1/8) Page 1
[INFO] (2/8) Page 2
[INFO] (3/8) Page 3
[INFO] (4/8) Page 4
[INFO] (5/8) Page 5
[INFO] (6/8) Page 6
[INFO] (7/8) Page 7
[INFO] (8/8) Page 8
[INFO] [4/4] Creating pages...
[INFO] (1/8) Page 1
[INFO] (2/8) Page 2
[INFO] (3/8) Page 3
[INFO] (4/8) Page 4
[INFO] (5/8) Page 5
[INFO] (6/8) Page 6
[INFO] (7/8) Page 7
[INFO] (8/8) Page 8
[INFO] Terminated in 17.27s.
[INFO] Start to convert ./pdfs\Abbas_2020.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing docu

In [5]:
#function for converting docx file to txt

def write_to_txt_file(file_name, data_list):
    with open(file_name, 'w', encoding='utf-8') as file:
        for item in data_list:
            file.write(item + '\n')

In [6]:
# removing figures, tables and in-text citations from the papers

def table_removal(doc_file):
    content = []
    doc = docx.Document(doc_file)
    patterns = [r'\[[0-9]+\]', r'\[[0-9]+,\s*[0-9]+\]', r'\[[0-9]+-[0-9]+\]', r'\[[0-9]+(,\s*[0-9]+)+\]']
    for paragraph in doc.paragraphs:
        para = paragraph.text
        if 'References' in para:
            break
        else:
            for pattern in patterns:
                para = re.sub(pattern, '', para)
            content.append(para)
    txt_file = doc_file[:-5]+".txt"
    write_to_txt_file(txt_file, content)

In [7]:
#storing docx files in a separate folder

if os.path.exists('./pdftodocx'):
    shutil.rmtree('./pdftodocx')
    os.mkdir('./pdftodocx')
else:
    os.mkdir('./pdftodocx')
    
for file in glob.glob('./pdfs/*.docx'):
    shutil.move(file, './pdftodocx')

In [8]:
#storing txt files in a separate folder

for file in glob.glob('./pdftodocx/*.docx'):
    table_removal(file)

if os.path.exists('./docxtotxt'):
    shutil.rmtree('./docxtotxt')
    os.mkdir('./docxtotxt')
else:
    os.mkdir('./docxtotxt')
    
for file in glob.glob('./pdftodocx/*.txt'):
    shutil.move(file, './docxtotxt')

# Task 2

In [9]:
#importing modules and libraries

from nltk.tokenize import word_tokenize
import nltk
import random
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from transformers import BertTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [11]:
# Removing stop words, stemming, and lemmatizing from the txt files

vector_dataset = []
for file in glob.glob('./docxtotxt/*.txt'):
    print(f'Cleaning the file {file}')
    final_tokenids = []
    with open(file,'r', encoding="utf8") as f:
        tokens = word_tokenize(f.read())
        for token in tokens:
            stemmed = stemmer.stem(token)
            lemmatized = lemmatizer.lemmatize(stemmed)
            if lemmatized not in stop_words:
                final_tokenids.append(lemmatized)
    vector_dataset.append(" ".join(final_tokenids))

Cleaning the file ./docxtotxt\15_Nazneen.txt
Cleaning the file ./docxtotxt\1_Ramırez-Duque_.txt
Cleaning the file ./docxtotxt\22_Ouss_ASD.txt
Cleaning the file ./docxtotxt\Abbas_2018.txt
Cleaning the file ./docxtotxt\Abbas_2020.txt
Cleaning the file ./docxtotxt\Asd_Cry_patterns.txt
Cleaning the file ./docxtotxt\carpenter2020 (1).txt
Cleaning the file ./docxtotxt\Dawson.txt
Cleaning the file ./docxtotxt\LEE.txt
Cleaning the file ./docxtotxt\Patten_Audio.txt
Cleaning the file ./docxtotxt\Qiu.txt
Cleaning the file ./docxtotxt\Tariq2018.txt
Cleaning the file ./docxtotxt\Tariq_2019.txt
Cleaning the file ./docxtotxt\Young_Behavior.txt
Cleaning the file ./docxtotxt\zhao2020.txt


In [12]:
print(len(vector_dataset))

15


In [13]:
# Defining a vectorizer and storing vector representation of each clean file in a vector database

vectorizer = TfidfVectorizer()
tfidf_database = vectorizer.fit_transform(vector_dataset)

In [14]:
# Vector database

print(tfidf_database)

  (1, 1203)	0.004198583023602918
  (1, 4124)	0.00463169509425807
  (1, 1483)	0.005190073668750274
  (1, 5)	0.00463169509425807
  (1, 1862)	0.005977064068055437
  (1, 943)	0.005977064068055437
  (1, 879)	0.005977064068055437
  (1, 4218)	0.005190073668750274
  (1, 4844)	0.005977064068055437
  (1, 3296)	0.005977064068055437
  (1, 3086)	0.005977064068055437
  (1, 604)	0.005977064068055437
  (1, 1349)	0.010636514493837533
  (1, 600)	0.005977064068055437
  (1, 1252)	0.005977064068055437
  (1, 3946)	0.005977064068055437
  (1, 3839)	0.005977064068055437
  (1, 3461)	0.0035455048312791775
  (1, 2563)	0.005977064068055437
  (1, 717)	0.005190073668750274
  (1, 560)	0.005190073668750274
  (1, 2578)	0.005977064068055437
  (1, 2033)	0.005190073668750274
  (1, 457)	0.003057714295647743
  (1, 1980)	0.005977064068055437
  :	:
  (14, 3278)	0.0018873283965060482
  (14, 1447)	0.07171847906722983
  (14, 3281)	0.0028190880059670675
  (14, 1435)	0.0018873283965060482
  (14, 1584)	0.01608574464343564
  (14, 19

# Task 3

In [15]:
#importing modules and libraries

from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

[INFO] NumExpr defaulting to 8 threads.


In [16]:
# Reading and storing the 13 questions from the csv file

questions = pd.read_csv("Query Questions - Sheet1.csv")
queries = list(questions.iloc[:, 1].values)
queries

['What are the variety of Multimodal and Multi-modular AI Approaches to Streamline Autism Diagnosis in Young Children',
 'What is Autism Spectrum Disorder, how it is caysed?',
 'What is the cure of Autism Spectrum Disorder',
 'What are Stereotypical and maladaptive behaviors in Autism Spectrum, how are these detected and managed',
 'How relevant is eye contact and how it can be used to detect Autism',
 'How can cross country trials help in development of Machine learning based Multimodal solutions ',
 'How early infants cry can help in the early detection of Autism ',
 'What are various methods to detect  Atypical Pattern of Facial expression in Children ',
 'What kind of facial expressions can be used to detect Autism Disorder in children',
 'What are methods to detect Autism from home videos ',
 'What is Still-Face Paradigm in Early Screening for High-Risk Autism Spectrum Disorder',
 'What is West Syndrome? ',
 'What is the utility of Behavior and interaction imaging at 9 months of a

In [17]:
# Converting each query into vector, then comparing with vector database using cosine_similarity to find the top 5 most similar research findings among the papers 
# printing the file index along with respective query 

k = 5
query_top_5 = []
for query in queries:
    query_vector = vectorizer.transform([query])
    query_similarities = cosine_similarity(query_vector, tfidf_database)
    top_k_indices = query_similarities.argsort()[0][-k:][::-1]
    print(top_k_indices, query)
    query_top_5.append(top_k_indices)

[11  4  3  7 10] What are the variety of Multimodal and Multi-modular AI Approaches to Streamline Autism Diagnosis in Young Children
[11  3  4 14 10] What is Autism Spectrum Disorder, how it is caysed?
[11  3  4 14 10] What is the cure of Autism Spectrum Disorder
[11  7  3 14  4] What are Stereotypical and maladaptive behaviors in Autism Spectrum, how are these detected and managed
[10  3 11  7  1] How relevant is eye contact and how it can be used to detect Autism
[3 8 1 6 4] How can cross country trials help in development of Machine learning based Multimodal solutions 
[11  3 14  4  5] How early infants cry can help in the early detection of Autism 
[14  6  1  7 11] What are various methods to detect  Atypical Pattern of Facial expression in Children 
[14  6  3 11  7] What kind of facial expressions can be used to detect Autism Disorder in children
[11  3 14  4  7] What are methods to detect Autism from home videos 
[10 11  3 14  1] What is Still-Face Paradigm in Early Screening for

In [18]:
len(query_top_5)

13

# Task 4

In [22]:
#importing modules and libraries

from langchain.chains.summarize import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import warnings
warnings.filterwarnings("ignore")

In [23]:
# Defining a summarizer using LLM OpenAI

summary_prompt = PromptTemplate(
    template="Please provide a concise summary of the following documents:\n\n{text}",
    input_variables=["text"]
)
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
llm = OpenAI(temperature=0.2)
summarizer = LLMChain(llm=llm, prompt=summary_prompt)

In [24]:
# Finding summaries for each query from the top 5 research findings of each query respectively and storing them in a list in order.
# Summary for a query can be retrieved from the list summaries if given the correct index number of the required query as stored in the list queries.

summaries = []
for query in query_top_5:
    documents = []
    for i in query:
        document = glob.glob('./docxtotxt/*.txt')[i-1]
        documents.append(document)
    summary = summarizer.run(documents)
    summaries.append(summary)

[INFO] HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 429 Too Many Requests"
[INFO] Retrying request to /completions in 0.825213 seconds
[INFO] HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 429 Too Many Requests"
[INFO] Retrying request to /completions in 1.828508 seconds
[INFO] HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 429 Too Many Requests"


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

'error': {'message': 'You exceeded your current quota, please check your plan and billing details. 
I used free OpenAI API which is not providing required quota, code is correct, will be running correctly with OpenAI paid version.